# Introduction to Monte Carlo Contracts

Python package [Monte Carlo Contracts](https://github.com/luphord/monte-carlo-contracts) provides composable financial contracts with Monte Carlo valuation.
This library employs ideas from [How to Write a Financial Contract](https://citeseerx.ist.psu.edu/viewdoc/summary?doi=10.1.1.14.7885) by S. L. Peyton Jones and J-M. Eber.
The core idea is that complex structured financial products can be *composed from a small set of elementary contracts*.
There are three central concepts to be understood:

1. Composable contracts
2. Observables
3. Acquisition dates

In [1]:
import mcc

In [2]:
from mcc import _contracts_table, _boolean_observables_table, _float_observables_table

In [3]:
from IPython.display import display, Markdown

## Composable Contracts

Composable contracts are the elementary building blocks for complex structured financial products.
In this Python library, all contracts are represented as classes deriving from abstract base class `Contract`.
The following contracts are available:

In [4]:
Markdown(_contracts_table(descr_col_width=150))

| Contract  | Description                                                                                                                                            |
|-----------|--------------------------------------------------------------------------------------------------------------------------------------------------------|
| Contract  | Abstract base class for all contracts                                                                                                                  |
| Zero      | Neither receive nor pay anything                                                                                                                       |
| One       | Receive one unit of currency at acquisition                                                                                                            |
| Give      | Receive all obligations of the underlying contract and pay all rights, i.e. invert the underlying contract                                             |
| And       | Obtain rights and obligations of all underlying contracts                                                                                              |
| Or        | Choose at acquisition between the underlying contracts                                                                                                 |
| Cond      | If observable is True at acquisition, obtain contract1, otherwise contract2                                                                            |
| Scale     | Same as the underling contract, but all payments scaled by the value of observable at acquisition                                                      |
| When      | Obtain the underlying contract as soon as observable becomes True after acquisition                                                                    |
| Delay     | Obtain the underlying contract and delay all payments to first occurence of observable.                                                                |
| Anytime   | At any point in time after acquisition when observable is True, choose whether to obtain the underlying contract or not; can be exercised only once    |
| Until     | Obtain the underlying contract, but as soon as observable becomes True after acquisition all following payments are nullified                          |
| Exchange  | Exchange cashflows resulting from contract to currency at the current spot rate                                                                        |

In [5]:
from mcc import (
    Contract,
    Zero,
    One,
    Give,
    And,
    Or,
    Cond,
    Scale,
    When,
    Delay,
    Anytime,
    Until,
    Exchange,
)

For example, the contract to exchange 1 EUR for 1 USD would be created as

In [6]:
And(Give(One("EUR")), One("USD"))

And(contracts=[Give(contract=One(currency='EUR')), One(currency='USD')])

With a bit of **syntactic sugar**, the same contract can be expressed as

In [7]:
-One("EUR") + One("USD")

And(contracts=[Give(contract=One(currency='EUR')), One(currency='USD')])

Of course, you can write **functions generating contracts** based on their arguments:

In [8]:
def exchange_coins(give_currency: str, receive_currency: str) -> Contract:
    return -One(give_currency) + One(receive_currency)

In [9]:
exchange_coins("EUR", "USD")

And(contracts=[Give(contract=One(currency='EUR')), One(currency='USD')])

In order to build more complex structures, you may also want to create **classes holding parameters for contract generation**.
In this case, you may want to implement the `ResolvableContract` abstract base class.

In [10]:
from dataclasses import dataclass
from mcc import ResolvableContract

In [11]:
@dataclass
class ExchangeCoins(ResolvableContract):
    give_currency: str
    receive_currency: str

    def resolve(self):
        return -One(self.give_currency) + One(self.receive_currency)

In [12]:
ExchangeCoins("EUR", "USD")

ExchangeCoins(give_currency='EUR', receive_currency='USD')

In [13]:
ExchangeCoins("EUR", "USD").resolve()

And(contracts=[Give(contract=One(currency='EUR')), One(currency='USD')])

Contracts allow you to define what cashflows are part of a product.
However, contracts on their own cannot express things like paying an amount of money determined by the closing price of a stock.
For that, you need *observables*.

## Observables

In [14]:
Markdown(_boolean_observables_table())

| Boolean Observable  | Description                                                            |
|---------------------|------------------------------------------------------------------------|
| ObservableBool      | Abstract base class for all observables of underlying type bool        |
| Not                 | True if observable is False and vice versa                             |
| AndObservable       | True if and only if both observables are True                          |
| OrObservable        | True if either or both observable are True                             |
| GreaterOrEqualThan  | True if and only if observable1 is greater or equal than observable2   |
| GreaterThan         | True if and only if observable1 is strictly greater than observable2   |
| At                  | True only at date                                                      |

In [15]:
Markdown(_float_observables_table(descr_col_width=135))

| Float Observable | Description                                                                                                                             |
|------------------|-----------------------------------------------------------------------------------------------------------------------------------------|
| ObservableFloat  | Abstract base class for all observables of underlying type float, essentially a real-valued stochastic process                          |
| Sum              | Equal to the sum of two observables                                                                                                     |
| Minus            | Negative value of observable                                                                                                            |
| Product          | Equal to the product (multiplication) of two observables                                                                                |
| Quotient         | Equal to the quotient (division) of two observables                                                                                     |
| Power            | Equal to observable1 to the power of observable2                                                                                        |
| Maximum          | Equal to the maximum of two observables                                                                                                 |
| Minimum          | Equal to the minimum of two observables                                                                                                 |
| RunningMax       | Running maximum of observable over time, seen from first_observation_idx.                                                               |
| RunningMin       | Running minimum of observable over time, seen from first_observation_idx.                                                               |
| FixedAfter       | Equal to observable, but remains constant as soon as fixing_condition becomes true after (including) first_observation_idx.             |
| Stock            | Value of the stock identified by identifier                                                                                             |
| FX               | Value of the currency spot between base_currency and counter_currency, i.e. 'one unit counter_currency' / 'one unit of base_currency'   |
| LinearRate       | Value of the linear rate (e.g. a LIBOR) with payment frequency in currency                                                              |
| KonstFloat       | Always equal to constant                                                                                                                |

## Acquisition Dates

## Displaying Help

### Print Help in the REPL

In [16]:
?mcc

Type:        module
String form: <module 'mcc' from '/home/luphord/repos/monte-carlo-contracts/mcc/__init__.py'>
File:        ~/repos/monte-carlo-contracts/mcc/__init__.py
Docstring:  
Composable financial contracts with Monte Carlo valuation


| Contract  | Description                                                                      |
|-----------|----------------------------------------------------------------------------------|
| Contract  | Abstract base class for all contracts                                            |
| Zero      | Neither receive nor pay anything                                                 |
| One       | Receive one unit of currency at acquisition                                      |
| Give      | Receive all obligations of the underlying contract and pay all rights, i.e. inve |
| And       | Obtain rights and obligations of all underlying contracts                        |
| Or        | Choose at acquisition between the underlying contracts        

### Generate Raw Markdown

In [17]:
print(_contracts_table(descr_col_width=150))
print()
print(_boolean_observables_table())
print()
print(_float_observables_table(descr_col_width=135))

| Contract  | Description                                                                                                                                            |
|-----------|--------------------------------------------------------------------------------------------------------------------------------------------------------|
| Contract  | Abstract base class for all contracts                                                                                                                  |
| Zero      | Neither receive nor pay anything                                                                                                                       |
| One       | Receive one unit of currency at acquisition                                                                                                            |
| Give      | Receive all obligations of the underlying contract and pay all rights, i.e. invert the underlying contract                                             